<a href="https://colab.research.google.com/github/kwisetree/AIFFEL_quest_cr/blob/main/Exploration/SubQuestC26/SubQuestC26_AIHUB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#프로젝트: 한국어 데이터로 챗봇 만들기 (AI HUB 개선된 버전)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###Step 1. 데이터 수집하기

In [2]:
# 로컬 파일을 코랩에 업로드하는 코드
from google.colab import files
import os

# 업로드 폴더 생성
upload_dir = '/content/json_files'
os.makedirs(upload_dir, exist_ok=True)

# 파일 업로드 (여러 파일 선택 가능)
uploaded = files.upload()

# 업로드한 파일을 지정된 디렉토리로 이동
for filename in uploaded.keys():
    os.rename(filename, os.path.join(upload_dir, filename))
    print(f'파일 {filename}이(가) {upload_dir}로 이동되었습니다.')

Saving DAILY-003794-AP600003-WZ000002-02_03_04-S2.json to DAILY-003794-AP600003-WZ000002-02_03_04-S2.json
Saving DAILY-003802-AP600004-WZ000003-02_03_09-S2.json to DAILY-003802-AP600004-WZ000003-02_03_09-S2.json
Saving DAILY-003809-AP600005-WZ000004-02_03_05-S2.json to DAILY-003809-AP600005-WZ000004-02_03_05-S2.json
Saving DAILY-003819-AP600007-WZ000005-02_03_02-S2.json to DAILY-003819-AP600007-WZ000005-02_03_02-S2.json
Saving DAILY-003830-AP600008-WZ000006-02_03_02-S2.json to DAILY-003830-AP600008-WZ000006-02_03_02-S2.json
Saving DAILY-003953-AP600009-WZ000005-02_03_02-S2.json to DAILY-003953-AP600009-WZ000005-02_03_02-S2.json
Saving DAILY-004160-AP600011-WZ000005-02_03_02-S2.json to DAILY-004160-AP600011-WZ000005-02_03_02-S2.json
Saving DAILY-004317-AP600015-WZ000005-02_03_02-S2.json to DAILY-004317-AP600015-WZ000005-02_03_02-S2.json
Saving DAILY-004341-AP600017-WZ000005-02_03_02-S2.json to DAILY-004341-AP600017-WZ000005-02_03_02-S2.json
Saving DAILY-004344-AP600019-WZ000006-02_03_02

In [3]:
# 업로드된 디렉토리와 파일 확인
import os

# 디렉토리 확인
upload_dir = '/content/json_files'
if not os.path.exists(upload_dir):
    print(f"디렉토리 {upload_dir}가 존재하지 않습니다. 생성합니다.")
    os.makedirs(upload_dir, exist_ok=True)
else:
    print(f"디렉토리 {upload_dir}가 존재합니다.")

    # 디렉토리 내 파일 목록 확인
    files = os.listdir(upload_dir)
    if files:
        print(f"디렉토리 내 파일 목록 ({len(files)}개):")
        for file in files:
            print(f" - {file}")
    else:
        print("디렉토리가 비어 있습니다.")

디렉토리 /content/json_files가 존재합니다.
디렉토리 내 파일 목록 (50개):
 - DAILY-004396-AP600025-WZ000006-02_03_02-S2.json
 - DAILY-004824-AP600053-WZ000005-02_03_02-S2.json
 - DAILY-004384-AP600024-WZ000005-02_03_02-S2.json
 - DAILY-005405-AP600062-WZ000005-02_03_02-S2.json
 - DAILY-005510-AP600073-WZ000010-02_03_10-S2.json
 - DAILY-005530-AP600075-WZ000005-02_03_02-S2.json
 - DAILY-004341-AP600017-WZ000005-02_03_02-S2.json
 - DAILY-004727-AP600041-WZ000001-01_05_10-S2.json
 - DAILY-005625-AP600080-WZ000005-02_03_02-S2.json
 - DAILY-004847-AP600056-WZ000005-02_03_02-S2.json
 - DAILY-004795-AP600050-WZ000009-02_03_04-S2.json
 - DAILY-004837-AP600055-WZ000005-02_03_02-S2.json
 - DAILY-005870-AP600099-WZ000001-02_04_10-S2.json
 - DAILY-004376-AP600022-WZ000005-02_03_02-S2.json
 - DAILY-004507-AP600032-WZ000006-02_03_02-S2.json
 - DAILY-004811-AP600052-WZ000005-02_03_02-S2.json
 - DAILY-003830-AP600008-WZ000006-02_03_02-S2.json
 - DAILY-005865-AP600097-WZ000001-02_04_10-S2.json
 - DAILY-005387-AP600061-WZ00

In [6]:
import json
import os
import pandas as pd

def extract_dialog_pairs(json_data):
    """
    JSON 데이터에서 대화 쌍을 추출하여 Q(질문)과 A(답변) 형태로 반환합니다.
    """
    dialog_pairs = []

    # 모든 세션을 처리
    for session in json_data["sessionInfo"]:
        dialog = session["dialog"]

        # 대화 쌍을 추출 (speaker1 = Q, speaker2 = A)
        for i in range(0, len(dialog) - 1, 2):  # 2단계씩 증가 (질문-답변 쌍)
            if i + 1 < len(dialog):  # 답변이 존재하는지 확인
                if dialog[i]["speaker"] == "speaker1" and dialog[i+1]["speaker"] == "speaker2":
                    question = dialog[i]["utterance"]
                    answer = dialog[i+1]["utterance"]
                    dialog_pairs.append({"Q": question, "A": answer})

    return dialog_pairs

def process_directory(directory_path):
    """
    디렉토리 내의 모든 JSON 파일을 처리합니다.
    """
    all_dialog_pairs = []

    # 디렉토리 내 모든 파일을 처리
    for filename in os.listdir(directory_path):
        if filename.endswith('.json'):
            file_path = os.path.join(directory_path, filename)

            # JSON 파일 로드
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    json_data = json.load(file)

                    # 대화 쌍 추출
                    dialog_pairs = extract_dialog_pairs(json_data)
                    all_dialog_pairs.extend(dialog_pairs)
                    print(f"파일 {filename} 처리 완료: {len(dialog_pairs)}개 대화 쌍 추출")
            except Exception as e:
                print(f"파일 {filename} 처리 중 오류 발생: {str(e)}")

    return all_dialog_pairs

# 메인 실행 코드
json_dir = '/content/json_files'  # JSON 파일이 있는 디렉토리
output_path = '/content/chat_data.csv'  # 출력 CSV 파일 경로

# 모든 JSON 파일 처리
dialog_pairs = process_directory(json_dir)

# 결과 확인
if dialog_pairs:
    df = pd.DataFrame(dialog_pairs)
    print(f"\n총 {len(df)}개의 대화 쌍이 추출되었습니다.")

    # 데이터 미리보기
    print("\n데이터 미리보기:")
    print(df.head())

    # CSV 파일로 저장
    df.to_csv(output_path, index=False, encoding='utf-8')
    print(f"\n대화 데이터가 {output_path}에 저장되었습니다.")
else:
    print("추출된 대화 쌍이 없습니다.")

# 추출한 CSV 파일 다운로드
from google.colab import files
files.download(output_path)

파일 DAILY-004396-AP600025-WZ000006-02_03_02-S2.json 처리 완료: 14개 대화 쌍 추출
파일 DAILY-004824-AP600053-WZ000005-02_03_02-S2.json 처리 완료: 14개 대화 쌍 추출
파일 DAILY-004384-AP600024-WZ000005-02_03_02-S2.json 처리 완료: 14개 대화 쌍 추출
파일 DAILY-005405-AP600062-WZ000005-02_03_02-S2.json 처리 완료: 14개 대화 쌍 추출
파일 DAILY-005510-AP600073-WZ000010-02_03_10-S2.json 처리 완료: 14개 대화 쌍 추출
파일 DAILY-005530-AP600075-WZ000005-02_03_02-S2.json 처리 완료: 14개 대화 쌍 추출
파일 DAILY-004341-AP600017-WZ000005-02_03_02-S2.json 처리 완료: 14개 대화 쌍 추출
파일 DAILY-004727-AP600041-WZ000001-01_05_10-S2.json 처리 완료: 14개 대화 쌍 추출
파일 DAILY-005625-AP600080-WZ000005-02_03_02-S2.json 처리 완료: 14개 대화 쌍 추출
파일 DAILY-004847-AP600056-WZ000005-02_03_02-S2.json 처리 완료: 14개 대화 쌍 추출
파일 DAILY-004795-AP600050-WZ000009-02_03_04-S2.json 처리 완료: 14개 대화 쌍 추출
파일 DAILY-004837-AP600055-WZ000005-02_03_02-S2.json 처리 완료: 14개 대화 쌍 추출
파일 DAILY-005870-AP600099-WZ000001-02_04_10-S2.json 처리 완료: 14개 대화 쌍 추출
파일 DAILY-004376-AP600022-WZ000005-02_03_02-S2.json 처리 완료: 14개 대화 쌍 추출
파일 DAILY-004507-AP60

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
# 저장된 경로에서 다시 불러오기
save_path = '/content/chat_data.csv'
df_check = pd.read_csv(save_path)
df_check.head()

,Q,A
0,안녕? 관광/ 감상의 유적에 대해서 이해를 높이고 있던 중에 궁금한 점이 생겨서 왔어!,안녕하세요! 저는 유적에 대해 잘 알고 있으니 무엇이든 물어봐주세요!
1,"내가 유적에 대한 책을 읽고 있었는데, 낙덕정에 대해 궁금해졌어. 혹시 낙덕정에 대...",그럼요. 낙덕정(樂德亭)은 전라북도 순창군 복흥면 상송리에 있는 정자예요.
2,오 전라북도에 있는 정자구나. 전북에 있는지는 몰랐어.,네. 그렇습니다. 낙덕정은 전라북도에 위치한 정자예요. 기회가 된다면 정자에서 쉬는...
3,맞아. 요즘 날씨가 너무 좋아서 정자에서 쉬면 기분이 좋을 것 같아.,네. 요즘 가을날씨라서 그런지 밖에 날씨가 정말 좋네요.
4,응응 나는 가을 날씨를 정말 좋아해. 나는 수영을 좋아해서 겨울이 오기 전에 수영을...,지금 날씨에 수영 다니기 좋을 것 같네요! 저도 수영하고 싶네요.


###Step 2. 데이터 전처리하기
---
영어 데이터와는 전혀 다른 데이터인 만큼 영어 데이터에 사용했던 전처리와 일부 동일한 전처리도 필요하겠지만 전체적으로는 다른 전처리를 수행해야 할 수도 있습니다.


In [9]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd

In [10]:
# 전처리 함수: 간단한 특수문자 제거 + 공백 정리 (숫자 유지)
def preprocess_sentence(sentence):
    sentence = str(sentence).strip().lower()  # 소문자화 + 양쪽 공백 제거

    # 구두점(punctuation) 주변에 공백 추가
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)

    # 여러 공백은 하나로
    sentence = re.sub(r'[" "]+', " ", sentence)

    # 한글, 영어, 숫자, 주요 구두점만 남기고 나머지 제거
    sentence = re.sub(r"[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9?.!,]+", " ", sentence)

    sentence = sentence.strip()
    return sentence

In [11]:
# 질문과 대답 각각 전처리 적용
questions = [preprocess_sentence(q) for q in df['Q']]
answers = [preprocess_sentence(a) for a in df['A']]

# 확인
for i in range(5):
    print(f"Q: {questions[i]}")
    print(f"A: {answers[i]}")
    print()

Q: 안녕 ? 관광 감상의 유적에 대해서 이해를 높이고 있던 중에 궁금한 점이 생겨서 왔어 !
A: 안녕하세요 ! 저는 유적에 대해 잘 알고 있으니 무엇이든 물어봐주세요 !

Q: 내가 유적에 대한 책을 읽고 있었는데 , 낙덕정에 대해 궁금해졌어 . 혹시 낙덕정에 대해 알려줄 수 있을까 ?
A: 그럼요 . 낙덕정 은 전라북도 순창군 복흥면 상송리에 있는 정자예요 .

Q: 오 전라북도에 있는 정자구나 . 전북에 있는지는 몰랐어 .
A: 네 . 그렇습니다 . 낙덕정은 전라북도에 위치한 정자예요 . 기회가 된다면 정자에서 쉬는 것도 좋을 것 같네요 .

Q: 맞아 . 요즘 날씨가 너무 좋아서 정자에서 쉬면 기분이 좋을 것 같아 .
A: 네 . 요즘 가을날씨라서 그런지 밖에 날씨가 정말 좋네요 .

Q: 응응 나는 가을 날씨를 정말 좋아해 . 나는 수영을 좋아해서 겨울이 오기 전에 수영을 많이 다닐 예정이야 .
A: 지금 날씨에 수영 다니기 좋을 것 같네요 ! 저도 수영하고 싶네요 .



###Step 3. SubwordTextEncoder 사용하기
---
한국어 데이터는 형태소 분석기를 사용하여 토크나이징을 해야 한다고 많은 분이 알고 있습니다. 하지만 여기서는 형태소 분석기가 아닌 위 실습에서 사용했던 내부 단어 토크나이저인 SubwordTextEncoder를 그대로 사용해보세요.


In [12]:
# 모든 문장을 하나로 합쳐 단어장 생성용 코퍼스 만들기
corpus = questions + answers

In [13]:
# SubwordTextEncoder 생성
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    corpus, target_vocab_size=2**14)

# 시작 토큰과 종료 토큰의 ID
START_TOKEN_ID = tokenizer.vocab_size
END_TOKEN_ID = tokenizer.vocab_size + 1

# 단어장 크기 (추가 토큰 포함)
VOCAB_SIZE = tokenizer.vocab_size + 2

print(f'단어장 크기: {VOCAB_SIZE}')

단어장 크기: 6246


In [14]:
# 문장을 토큰화하는 함수
def tokenize_and_filter(inputs, outputs, max_length=40):
    tokenized_inputs, tokenized_outputs = [], []

    for (input_sentence, output_sentence) in zip(inputs, outputs):
        # 각 문장을 정수 시퀀스로 변환
        input_sentence_tokens = [START_TOKEN_ID] + tokenizer.encode(input_sentence) + [END_TOKEN_ID]
        output_sentence_tokens = [START_TOKEN_ID] + tokenizer.encode(output_sentence) + [END_TOKEN_ID]

        # 최대 길이 체크
        if len(input_sentence_tokens) <= max_length and len(output_sentence_tokens) <= max_length:
            tokenized_inputs.append(input_sentence_tokens)
            tokenized_outputs.append(output_sentence_tokens)

    # 패딩 추가
    tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
        tokenized_inputs, maxlen=max_length, padding='post')
    tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
        tokenized_outputs, maxlen=max_length, padding='post')

    return tokenized_inputs, tokenized_outputs

In [42]:
# 토큰화 및 필터링 적용
MAX_LENGTH = 300
questions_tokens, answers_tokens = tokenize_and_filter(questions, answers, MAX_LENGTH)

print(f'질문 토큰화 결과 크기: {len(questions_tokens)}')
print(f'대답 토큰화 결과 크기: {len(answers_tokens)}')

질문 토큰화 결과 크기: 700
대답 토큰화 결과 크기: 700


In [43]:
# 데이터셋 생성
BATCH_SIZE = 64
BUFFER_SIZE = 20000

# 디코더 입력과 타겟 구분 (교사 강요를 위한 준비)
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions_tokens,
        'dec_inputs': answers_tokens[:, :-1]  # 마지막 토큰 제외
    },
    {
        'outputs': answers_tokens[:, 1:]  # 첫 번째 토큰 제외
    }
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

print(f'데이터셋: {dataset}')

데이터셋: <_PrefetchDataset element_spec=({'inputs': TensorSpec(shape=(None, 300), dtype=tf.int32, name=None), 'dec_inputs': TensorSpec(shape=(None, 299), dtype=tf.int32, name=None)}, {'outputs': TensorSpec(shape=(None, 299), dtype=tf.int32, name=None)})>


###Step 4. 모델 구성하기
---
위 실습 내용을 참고하여 트랜스포머 모델을 구현합니다.

1. 포지셔널 인코딩

In [44]:
# 위치 인코딩
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                            np.arange(d_model)[np.newaxis, :],
                            d_model)

    # sin 함수는 짝수 인덱스에 적용
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # cos 함수는 홀수 인덱스에 적용
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

2. 스케일드 닷 프로덕트 어텐션

In [45]:
# 어텐션 마스킹 함수
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (seq_len, seq_len)

In [46]:
# 스케일드 닷 프로덕트 어텐션
def scaled_dot_product_attention(q, k, v, mask):
    # Q와 K의 곱셈 연산
    matmul_qk = tf.matmul(q, k, transpose_b=True)

    # 스케일링
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # 마스킹 적용
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)

    # 소프트맥스로 어텐션 가중치 계산
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

    # 가중치와 V 곱하기
    output = tf.matmul(attention_weights, v)

    return output, attention_weights

3. 멀티헤드 어텐션

In [47]:
# 멀티헤드 어텐션 레이어
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # 스케일드 닷 프로덕트 어텐션 적용
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights

4. 포인트 와이즈 피드포워드 네트워크

In [48]:
# 포지션 와이즈 피드 포워드 네트워크
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
        tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
    ])

5. 인코더 레이어

In [49]:
# 인코더 레이어
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask):
        attn_output, _ = self.mha(v=x, k=x, q=x, mask=mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)

        return out2

6. 디코더 레이어

In [50]:
# 디코더 레이어
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)

    def call(self, x, enc_output, training,
           look_ahead_mask, padding_mask):
        # enc_output.shape == (batch_size, input_seq_len, d_model)

        attn1, attn_weights_block1 = self.mha1(v=x, k=x, q=x, mask=look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)

        attn2, attn_weights_block2 = self.mha2(
            v=enc_output, k=enc_output, q=out1, mask=padding_mask)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)

        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)

        return out3, attn_weights_block1, attn_weights_block2

7. 전체 인코더

In [51]:
# 인코더
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               maximum_position_encoding, rate=0.1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding,
                                              self.d_model)

        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate)
                         for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask):
        seq_len = tf.shape(x)[1]

        # 입력을 임베딩
        x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x=x, training=training, mask=mask)

        return x  # (batch_size, input_seq_len, d_model)

8. 전체 디코더

In [52]:
# 디코더
class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size,
               maximum_position_encoding, rate=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate)
                         for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, enc_output, training,
           look_ahead_mask, padding_mask):
        seq_len = tf.shape(x)[1]
        attention_weights = {}

        x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](
                x=x,
                enc_output=enc_output,
                training=training,
                look_ahead_mask=look_ahead_mask,
                padding_mask=padding_mask
            )

            attention_weights[f'decoder_layer{i+1}_block1'] = block1
            attention_weights[f'decoder_layer{i+1}_block2'] = block2

        # x.shape == (batch_size, target_seq_len, d_model)
        return x, attention_weights

9. 트랜스포머 전체 모델

In [53]:
# 패딩 마스크 생성: 입력 시퀀스에서 패딩 토큰(0)에 해당하는 위치를 마스킹
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]  # (배치 크기, 1, 1, 시퀀스 길이)

# 미래 토큰 마스킹 (Look-ahead mask): 디코더가 미래의 토큰을 보지 못하게 막음
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (시퀀스 길이, 시퀀스 길이)

# 전체 마스크 생성 함수: 인코더/디코더 패딩 마스크와 Look-ahead 마스크 결합
def create_masks(inp, tar):
    # 인코더 입력에 대한 패딩 마스크
    enc_padding_mask = create_padding_mask(inp)

    # 디코더 입력에 대한 look-ahead 마스크와 패딩 마스크 결합
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    # 디코더의 인코더 출력에 적용할 패딩 마스크
    dec_padding_mask = create_padding_mask(inp)

    return enc_padding_mask, combined_mask, dec_padding_mask


In [54]:
# 트랜스포머 모델
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               target_vocab_size, pe_input, pe_target, rate=0.1):
        super(Transformer, self).__init__()

        self.encoder = Encoder(num_layers, d_model, num_heads, dff,
                             input_vocab_size, pe_input, rate)

        self.decoder = Decoder(num_layers, d_model, num_heads, dff,
                             target_vocab_size, pe_target, rate)

        self.final_layer = tf.keras.layers.Dense(target_vocab_size)

    def call(self, inputs, training=False):
        # Keras 모델은 inputs를 리스트나 딕셔너리로 받음
        inp, tar = inputs['inputs'], inputs['dec_inputs']

        enc_padding_mask, look_ahead_mask, dec_padding_mask = create_masks(inp, tar)

        enc_output = self.encoder(
            x=inp,
            training=training,
            mask=enc_padding_mask
        )

        dec_output, attention_weights = self.decoder(
            x=tar,
            enc_output=enc_output,
            training=training,
            look_ahead_mask=look_ahead_mask,
            padding_mask=dec_padding_mask
        )

        final_output = self.final_layer(dec_output)
        return final_output

10. 모델 생성

In [55]:
# 학습 파라미터 설정
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
dropout_rate = 0.1

# 모델 인스턴스 생성
transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=VOCAB_SIZE,
    target_vocab_size=VOCAB_SIZE,
    pe_input=MAX_LENGTH,
    pe_target=MAX_LENGTH,
    rate=dropout_rate
)

In [56]:
# 손실 함수 정의
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = tf.keras.losses.sparse_categorical_crossentropy(
        real, pred, from_logits=True)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_) / tf.reduce_sum(mask)

# 정확도 계산 함수 정의
def accuracy_function(real, pred):
    accuracies = tf.equal(real, tf.cast(tf.argmax(pred, axis=2), dtype=tf.int32))

    mask = tf.math.logical_not(tf.math.equal(real, 0))
    accuracies = tf.math.logical_and(mask, accuracies)

    accuracies = tf.cast(accuracies, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)

    return tf.reduce_sum(accuracies) / tf.reduce_sum(mask)

In [57]:
# 학습률 스케줄러 정의
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        # step을 float32로 명시적 변환 추가
        step = tf.cast(step, tf.float32)

        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

# 학습률 설정 및 옵티마이저 정의
learning_rate = CustomSchedule(d_model)
optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

# 체크포인트 설정
checkpoint_path = './checkpoints/transformer'
ckpt = tf.train.Checkpoint(transformer=transformer, optimizer=optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

In [59]:
# 학습 함수 정의
@tf.function
def train_step(inp, tar):
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]

    with tf.GradientTape() as tape:
        predictions = transformer(
            {
                'inputs': inp,
                'dec_inputs': tar_inp
            },
            training=True
        )

        loss = loss_function(tar_real, predictions)

    gradients = tape.gradient(loss, transformer.trainable_variables)
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    train_loss(loss)
    train_accuracy(accuracy_function(tar_real, predictions))

In [60]:
# tqdm 라이브러리 설치 및 가져오기
!pip install tqdm
from tqdm.notebook import tqdm

# 학습 메트릭 정의
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')

# 모델 학습
EPOCHS = 50

for epoch in range(EPOCHS):
    start = time.time()

    train_loss.reset_state()
    train_accuracy.reset_state()

    print(f"에포크 {epoch + 1}/{EPOCHS} 시작...")

    # tqdm으로 배치 순회를 감싸서 진행 표시줄 표시
    for (batch, (inp, tar)) in tqdm(enumerate(dataset), total=len(dataset)):
        train_step(inp['inputs'], tar['outputs'])

    # 에포크 완료 후 결과 출력
    epoch_time = time.time() - start
    print(f'에포크 {epoch + 1} 완료 - 손실: {train_loss.result():.4f}, 정확도: {train_accuracy.result():.4f}, 시간: {epoch_time:.2f}초')

    # 에포크가 끝날 때마다 체크포인트 저장
    if (epoch + 1) % 5 == 0:
        ckpt_save_path = ckpt_manager.save()
        print(f'에포크 {epoch + 1}에 체크포인트 저장: {ckpt_save_path}')

    print("-" * 50)

에포크 1/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 1 완료 - 손실: 8.7071, 정확도: 0.0001, 시간: 41.16초
--------------------------------------------------
에포크 2/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 2 완료 - 손실: 8.6892, 정확도: 0.0000, 시간: 5.19초
--------------------------------------------------
에포크 3/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 3 완료 - 손실: 8.6547, 정확도: 0.0110, 시간: 5.21초
--------------------------------------------------
에포크 4/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 4 완료 - 손실: 8.6115, 정확도: 0.0710, 시간: 5.22초
--------------------------------------------------
에포크 5/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 5 완료 - 손실: 8.5677, 정확도: 0.0886, 시간: 5.25초
에포크 5에 체크포인트 저장: ./checkpoints/transformer/ckpt-1
--------------------------------------------------
에포크 6/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 6 완료 - 손실: 8.5278, 정확도: 0.0892, 시간: 5.24초
--------------------------------------------------
에포크 7/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 7 완료 - 손실: 8.4901, 정확도: 0.0890, 시간: 5.25초
--------------------------------------------------
에포크 8/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 8 완료 - 손실: 8.4533, 정확도: 0.0891, 시간: 5.27초
--------------------------------------------------
에포크 9/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 9 완료 - 손실: 8.4162, 정확도: 0.0891, 시간: 5.28초
--------------------------------------------------
에포크 10/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 10 완료 - 손실: 8.3750, 정확도: 0.0891, 시간: 5.29초
에포크 10에 체크포인트 저장: ./checkpoints/transformer/ckpt-2
--------------------------------------------------
에포크 11/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 11 완료 - 손실: 8.3347, 정확도: 0.0890, 시간: 5.29초
--------------------------------------------------
에포크 12/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 12 완료 - 손실: 8.2886, 정확도: 0.0892, 시간: 5.32초
--------------------------------------------------
에포크 13/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 13 완료 - 손실: 8.2395, 정확도: 0.0894, 시간: 5.33초
--------------------------------------------------
에포크 14/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 14 완료 - 손실: 8.1890, 정확도: 0.0893, 시간: 5.34초
--------------------------------------------------
에포크 15/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 15 완료 - 손실: 8.1346, 정확도: 0.0891, 시간: 5.36초
에포크 15에 체크포인트 저장: ./checkpoints/transformer/ckpt-3
--------------------------------------------------
에포크 16/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 16 완료 - 손실: 8.0785, 정확도: 0.0893, 시간: 5.37초
--------------------------------------------------
에포크 17/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 17 완료 - 손실: 8.0177, 정확도: 0.0892, 시간: 5.39초
--------------------------------------------------
에포크 18/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 18 완료 - 손실: 7.9544, 정확도: 0.0894, 시간: 5.40초
--------------------------------------------------
에포크 19/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 19 완료 - 손실: 7.8889, 정확도: 0.0896, 시간: 5.42초
--------------------------------------------------
에포크 20/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 20 완료 - 손실: 7.8234, 정확도: 0.0891, 시간: 5.43초
에포크 20에 체크포인트 저장: ./checkpoints/transformer/ckpt-4
--------------------------------------------------
에포크 21/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 21 완료 - 손실: 7.7508, 정확도: 0.0898, 시간: 5.44초
--------------------------------------------------
에포크 22/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 22 완료 - 손실: 7.6787, 정확도: 0.0916, 시간: 5.45초
--------------------------------------------------
에포크 23/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 23 완료 - 손실: 7.6119, 정확도: 0.0931, 시간: 5.47초
--------------------------------------------------
에포크 24/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 24 완료 - 손실: 7.5345, 정확도: 0.0977, 시간: 5.47초
--------------------------------------------------
에포크 25/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 25 완료 - 손실: 7.4664, 정확도: 0.1009, 시간: 5.44초
에포크 25에 체크포인트 저장: ./checkpoints/transformer/ckpt-5
--------------------------------------------------
에포크 26/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 26 완료 - 손실: 7.3958, 정확도: 0.1076, 시간: 5.41초
--------------------------------------------------
에포크 27/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 27 완료 - 손실: 7.3227, 정확도: 0.1183, 시간: 5.42초
--------------------------------------------------
에포크 28/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 28 완료 - 손실: 7.2506, 정확도: 0.1228, 시간: 5.42초
--------------------------------------------------
에포크 29/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 29 완료 - 손실: 7.1852, 정확도: 0.1327, 시간: 5.41초
--------------------------------------------------
에포크 30/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 30 완료 - 손실: 7.1137, 정확도: 0.1420, 시간: 5.42초
에포크 30에 체크포인트 저장: ./checkpoints/transformer/ckpt-6
--------------------------------------------------
에포크 31/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 31 완료 - 손실: 7.0537, 정확도: 0.1432, 시간: 5.40초
--------------------------------------------------
에포크 32/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 32 완료 - 손실: 6.9870, 정확도: 0.1494, 시간: 5.42초
--------------------------------------------------
에포크 33/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 33 완료 - 손실: 6.9241, 정확도: 0.1502, 시간: 5.42초
--------------------------------------------------
에포크 34/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 34 완료 - 손실: 6.8575, 정확도: 0.1574, 시간: 5.43초
--------------------------------------------------
에포크 35/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 35 완료 - 손실: 6.7981, 정확도: 0.1575, 시간: 5.44초
에포크 35에 체크포인트 저장: ./checkpoints/transformer/ckpt-7
--------------------------------------------------
에포크 36/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 36 완료 - 손실: 6.7381, 정확도: 0.1599, 시간: 5.43초
--------------------------------------------------
에포크 37/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 37 완료 - 손실: 6.6721, 정확도: 0.1638, 시간: 5.45초
--------------------------------------------------
에포크 38/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 38 완료 - 손실: 6.6042, 정확도: 0.1687, 시간: 5.44초
--------------------------------------------------
에포크 39/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 39 완료 - 손실: 6.5425, 정확도: 0.1710, 시간: 5.47초
--------------------------------------------------
에포크 40/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 40 완료 - 손실: 6.4672, 정확도: 0.1735, 시간: 5.44초
에포크 40에 체크포인트 저장: ./checkpoints/transformer/ckpt-8
--------------------------------------------------
에포크 41/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 41 완료 - 손실: 6.3989, 정확도: 0.1759, 시간: 5.44초
--------------------------------------------------
에포크 42/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 42 완료 - 손실: 6.3304, 정확도: 0.1767, 시간: 5.47초
--------------------------------------------------
에포크 43/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 43 완료 - 손실: 6.2621, 정확도: 0.1770, 시간: 5.44초
--------------------------------------------------
에포크 44/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 44 완료 - 손실: 6.1780, 정확도: 0.1809, 시간: 5.45초
--------------------------------------------------
에포크 45/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 45 완료 - 손실: 6.1018, 정확도: 0.1835, 시간: 5.45초
에포크 45에 체크포인트 저장: ./checkpoints/transformer/ckpt-9
--------------------------------------------------
에포크 46/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 46 완료 - 손실: 6.0167, 정확도: 0.1884, 시간: 5.42초
--------------------------------------------------
에포크 47/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 47 완료 - 손실: 5.9504, 정확도: 0.1869, 시간: 5.43초
--------------------------------------------------
에포크 48/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 48 완료 - 손실: 5.8698, 정확도: 0.1904, 시간: 5.44초
--------------------------------------------------
에포크 49/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 49 완료 - 손실: 5.7845, 정확도: 0.1928, 시간: 5.44초
--------------------------------------------------
에포크 50/50 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 50 완료 - 손실: 5.7075, 정확도: 0.1973, 시간: 5.45초
에포크 50에 체크포인트 저장: ./checkpoints/transformer/ckpt-10
--------------------------------------------------


In [61]:
# 체크포인트 복원
ckpt.restore(ckpt_manager.latest_checkpoint)

if ckpt_manager.latest_checkpoint:
    print(f"체크포인트에서 복원됨: {ckpt_manager.latest_checkpoint}")
    start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
else:
    print("체크포인트가 없습니다. 처음부터 학습을 시작합니다.")
    start_epoch = 0

# 이어서 학습 (start_epoch부터 100까지)
EPOCHS = 100

for epoch in range(start_epoch, EPOCHS):
    start = time.time()

    train_loss.reset_state()
    train_accuracy.reset_state()

    print(f"에포크 {epoch + 1}/{EPOCHS} 시작...")

    for (batch, (inp, tar)) in tqdm(enumerate(dataset), total=len(dataset)):
        train_step(inp['inputs'], tar['outputs'])

    epoch_time = time.time() - start
    print(f'에포크 {epoch + 1} 완료 - 손실: {train_loss.result():.4f}, 정확도: {train_accuracy.result():.4f}, 시간: {epoch_time:.2f}초')

    if (epoch + 1) % 5 == 0:
        ckpt_save_path = ckpt_manager.save()
        print(f'에포크 {epoch + 1}에 체크포인트 저장: {ckpt_save_path}')

    print("-" * 50)


체크포인트에서 복원됨: ./checkpoints/transformer/ckpt-10
에포크 11/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 11 완료 - 손실: 5.6227, 정확도: 0.2060, 시간: 5.47초
--------------------------------------------------
에포크 12/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 12 완료 - 손실: 5.5398, 정확도: 0.2078, 시간: 5.44초
--------------------------------------------------
에포크 13/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 13 완료 - 손실: 5.4735, 정확도: 0.2110, 시간: 5.44초
--------------------------------------------------
에포크 14/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 14 완료 - 손실: 5.3901, 정확도: 0.2193, 시간: 5.45초
--------------------------------------------------
에포크 15/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 15 완료 - 손실: 5.3173, 정확도: 0.2230, 시간: 5.46초
에포크 15에 체크포인트 저장: ./checkpoints/transformer/ckpt-11
--------------------------------------------------
에포크 16/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 16 완료 - 손실: 5.2425, 정확도: 0.2278, 시간: 5.44초
--------------------------------------------------
에포크 17/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 17 완료 - 손실: 5.1677, 정확도: 0.2379, 시간: 5.46초
--------------------------------------------------
에포크 18/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 18 완료 - 손실: 5.0843, 정확도: 0.2428, 시간: 5.45초
--------------------------------------------------
에포크 19/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 19 완료 - 손실: 5.0137, 정확도: 0.2479, 시간: 5.47초
--------------------------------------------------
에포크 20/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 20 완료 - 손실: 4.9486, 정확도: 0.2545, 시간: 5.46초
에포크 20에 체크포인트 저장: ./checkpoints/transformer/ckpt-12
--------------------------------------------------
에포크 21/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 21 완료 - 손실: 4.8737, 정확도: 0.2600, 시간: 5.44초
--------------------------------------------------
에포크 22/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 22 완료 - 손실: 4.7893, 정확도: 0.2658, 시간: 5.44초
--------------------------------------------------
에포크 23/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 23 완료 - 손실: 4.7160, 정확도: 0.2690, 시간: 5.44초
--------------------------------------------------
에포크 24/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 24 완료 - 손실: 4.6468, 정확도: 0.2755, 시간: 5.45초
--------------------------------------------------
에포크 25/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 25 완료 - 손실: 4.5754, 정확도: 0.2792, 시간: 5.46초
에포크 25에 체크포인트 저장: ./checkpoints/transformer/ckpt-13
--------------------------------------------------
에포크 26/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 26 완료 - 손실: 4.5050, 정확도: 0.2858, 시간: 5.42초
--------------------------------------------------
에포크 27/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 27 완료 - 손실: 4.4349, 정확도: 0.2871, 시간: 5.44초
--------------------------------------------------
에포크 28/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 28 완료 - 손실: 4.3480, 정확도: 0.2981, 시간: 5.43초
--------------------------------------------------
에포크 29/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 29 완료 - 손실: 4.2762, 정확도: 0.3042, 시간: 5.44초
--------------------------------------------------
에포크 30/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 30 완료 - 손실: 4.2031, 정확도: 0.3094, 시간: 5.45초
에포크 30에 체크포인트 저장: ./checkpoints/transformer/ckpt-14
--------------------------------------------------
에포크 31/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 31 완료 - 손실: 4.1364, 정확도: 0.3143, 시간: 5.43초
--------------------------------------------------
에포크 32/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 32 완료 - 손실: 4.0658, 정확도: 0.3260, 시간: 5.46초
--------------------------------------------------
에포크 33/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 33 완료 - 손실: 3.9884, 정확도: 0.3321, 시간: 5.44초
--------------------------------------------------
에포크 34/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 34 완료 - 손실: 3.9092, 정확도: 0.3429, 시간: 5.45초
--------------------------------------------------
에포크 35/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 35 완료 - 손실: 3.8562, 정확도: 0.3495, 시간: 5.44초
에포크 35에 체크포인트 저장: ./checkpoints/transformer/ckpt-15
--------------------------------------------------
에포크 36/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 36 완료 - 손실: 3.7815, 정확도: 0.3615, 시간: 5.43초
--------------------------------------------------
에포크 37/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 37 완료 - 손실: 3.6975, 정확도: 0.3794, 시간: 5.45초
--------------------------------------------------
에포크 38/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 38 완료 - 손실: 3.6316, 정확도: 0.3917, 시간: 5.44초
--------------------------------------------------
에포크 39/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 39 완료 - 손실: 3.5611, 정확도: 0.4050, 시간: 5.45초
--------------------------------------------------
에포크 40/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 40 완료 - 손실: 3.4840, 정확도: 0.4190, 시간: 5.45초
에포크 40에 체크포인트 저장: ./checkpoints/transformer/ckpt-16
--------------------------------------------------
에포크 41/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 41 완료 - 손실: 3.4239, 정확도: 0.4357, 시간: 5.44초
--------------------------------------------------
에포크 42/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 42 완료 - 손실: 3.3406, 정확도: 0.4463, 시간: 5.45초
--------------------------------------------------
에포크 43/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 43 완료 - 손실: 3.2695, 정확도: 0.4676, 시간: 5.45초
--------------------------------------------------
에포크 44/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 44 완료 - 손실: 3.1962, 정확도: 0.4880, 시간: 5.46초
--------------------------------------------------
에포크 45/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 45 완료 - 손실: 3.1284, 정확도: 0.5012, 시간: 5.46초
에포크 45에 체크포인트 저장: ./checkpoints/transformer/ckpt-17
--------------------------------------------------
에포크 46/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 46 완료 - 손실: 3.0521, 정확도: 0.5196, 시간: 5.44초
--------------------------------------------------
에포크 47/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 47 완료 - 손실: 2.9737, 정확도: 0.5400, 시간: 5.45초
--------------------------------------------------
에포크 48/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 48 완료 - 손실: 2.9036, 정확도: 0.5632, 시간: 5.46초
--------------------------------------------------
에포크 49/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 49 완료 - 손실: 2.8284, 정확도: 0.5827, 시간: 5.46초
--------------------------------------------------
에포크 50/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 50 완료 - 손실: 2.7503, 정확도: 0.6057, 시간: 5.46초
에포크 50에 체크포인트 저장: ./checkpoints/transformer/ckpt-18
--------------------------------------------------
에포크 51/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 51 완료 - 손실: 2.6730, 정확도: 0.6220, 시간: 5.43초
--------------------------------------------------
에포크 52/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 52 완료 - 손실: 2.6054, 정확도: 0.6466, 시간: 5.45초
--------------------------------------------------
에포크 53/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 53 완료 - 손실: 2.5340, 정확도: 0.6644, 시간: 5.46초
--------------------------------------------------
에포크 54/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 54 완료 - 손실: 2.4526, 정확도: 0.6859, 시간: 5.45초
--------------------------------------------------
에포크 55/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 55 완료 - 손실: 2.3842, 정확도: 0.7010, 시간: 5.46초
에포크 55에 체크포인트 저장: ./checkpoints/transformer/ckpt-19
--------------------------------------------------
에포크 56/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 56 완료 - 손실: 2.3071, 정확도: 0.7254, 시간: 5.43초
--------------------------------------------------
에포크 57/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 57 완료 - 손실: 2.2426, 정확도: 0.7361, 시간: 5.47초
--------------------------------------------------
에포크 58/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 58 완료 - 손실: 2.1701, 정확도: 0.7520, 시간: 5.46초
--------------------------------------------------
에포크 59/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 59 완료 - 손실: 2.0944, 정확도: 0.7744, 시간: 5.45초
--------------------------------------------------
에포크 60/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 60 완료 - 손실: 2.0142, 정확도: 0.7875, 시간: 5.44초
에포크 60에 체크포인트 저장: ./checkpoints/transformer/ckpt-20
--------------------------------------------------
에포크 61/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 61 완료 - 손실: 1.9472, 정확도: 0.8016, 시간: 5.43초
--------------------------------------------------
에포크 62/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 62 완료 - 손실: 1.8732, 정확도: 0.8192, 시간: 5.46초
--------------------------------------------------
에포크 63/100 시작...


  0%|          | 0/11 [00:04<?, ?it/s]

에포크 63 완료 - 손실: 1.8079, 정확도: 0.8280, 시간: 10.15초
--------------------------------------------------
에포크 64/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 64 완료 - 손실: 1.7339, 정확도: 0.8368, 시간: 5.43초
--------------------------------------------------
에포크 65/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 65 완료 - 손실: 1.6690, 정확도: 0.8550, 시간: 5.45초
에포크 65에 체크포인트 저장: ./checkpoints/transformer/ckpt-21
--------------------------------------------------
에포크 66/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 66 완료 - 손실: 1.5904, 정확도: 0.8654, 시간: 5.45초
--------------------------------------------------
에포크 67/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 67 완료 - 손실: 1.5306, 정확도: 0.8757, 시간: 5.48초
--------------------------------------------------
에포크 68/100 시작...


  0%|          | 0/11 [00:04<?, ?it/s]

에포크 68 완료 - 손실: 1.4656, 정확도: 0.8822, 시간: 10.16초
--------------------------------------------------
에포크 69/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 69 완료 - 손실: 1.3961, 정확도: 0.8916, 시간: 5.44초
--------------------------------------------------
에포크 70/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 70 완료 - 손실: 1.3304, 정확도: 0.8993, 시간: 5.46초
에포크 70에 체크포인트 저장: ./checkpoints/transformer/ckpt-22
--------------------------------------------------
에포크 71/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 71 완료 - 손실: 1.2701, 정확도: 0.9097, 시간: 5.45초
--------------------------------------------------
에포크 72/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 72 완료 - 손실: 1.2065, 정확도: 0.9152, 시간: 5.46초
--------------------------------------------------
에포크 73/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 73 완료 - 손실: 1.1431, 정확도: 0.9194, 시간: 5.46초
--------------------------------------------------
에포크 74/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 74 완료 - 손실: 1.0862, 정확도: 0.9326, 시간: 5.46초
--------------------------------------------------
에포크 75/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 75 완료 - 손실: 1.0245, 정확도: 0.9359, 시간: 5.45초
에포크 75에 체크포인트 저장: ./checkpoints/transformer/ckpt-23
--------------------------------------------------
에포크 76/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 76 완료 - 손실: 0.9750, 정확도: 0.9430, 시간: 5.43초
--------------------------------------------------
에포크 77/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 77 완료 - 손실: 0.9120, 정확도: 0.9493, 시간: 5.44초
--------------------------------------------------
에포크 78/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 78 완료 - 손실: 0.8533, 정확도: 0.9530, 시간: 5.44초
--------------------------------------------------
에포크 79/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 79 완료 - 손실: 0.7994, 정확도: 0.9568, 시간: 5.43초
--------------------------------------------------
에포크 80/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 80 완료 - 손실: 0.7564, 정확도: 0.9626, 시간: 5.43초
에포크 80에 체크포인트 저장: ./checkpoints/transformer/ckpt-24
--------------------------------------------------
에포크 81/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 81 완료 - 손실: 0.7090, 정확도: 0.9673, 시간: 5.42초
--------------------------------------------------
에포크 82/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 82 완료 - 손실: 0.6624, 정확도: 0.9715, 시간: 5.45초
--------------------------------------------------
에포크 83/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 83 완료 - 손실: 0.6166, 정확도: 0.9746, 시간: 5.45초
--------------------------------------------------
에포크 84/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 84 완료 - 손실: 0.5775, 정확도: 0.9763, 시간: 5.46초
--------------------------------------------------
에포크 85/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 85 완료 - 손실: 0.5301, 정확도: 0.9791, 시간: 5.48초
에포크 85에 체크포인트 저장: ./checkpoints/transformer/ckpt-25
--------------------------------------------------
에포크 86/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 86 완료 - 손실: 0.4955, 정확도: 0.9815, 시간: 5.44초
--------------------------------------------------
에포크 87/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 87 완료 - 손실: 0.4581, 정확도: 0.9853, 시간: 5.48초
--------------------------------------------------
에포크 88/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 88 완료 - 손실: 0.4262, 정확도: 0.9871, 시간: 5.47초
--------------------------------------------------
에포크 89/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 89 완료 - 손실: 0.3890, 정확도: 0.9911, 시간: 5.46초
--------------------------------------------------
에포크 90/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 90 완료 - 손실: 0.3588, 정확도: 0.9916, 시간: 5.47초
에포크 90에 체크포인트 저장: ./checkpoints/transformer/ckpt-26
--------------------------------------------------
에포크 91/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 91 완료 - 손실: 0.3335, 정확도: 0.9923, 시간: 5.44초
--------------------------------------------------
에포크 92/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 92 완료 - 손실: 0.3062, 정확도: 0.9926, 시간: 5.46초
--------------------------------------------------
에포크 93/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 93 완료 - 손실: 0.2837, 정확도: 0.9945, 시간: 5.44초
--------------------------------------------------
에포크 94/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 94 완료 - 손실: 0.2616, 정확도: 0.9940, 시간: 5.44초
--------------------------------------------------
에포크 95/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 95 완료 - 손실: 0.2405, 정확도: 0.9956, 시간: 5.45초
에포크 95에 체크포인트 저장: ./checkpoints/transformer/ckpt-27
--------------------------------------------------
에포크 96/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 96 완료 - 손실: 0.2264, 정확도: 0.9954, 시간: 5.43초
--------------------------------------------------
에포크 97/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 97 완료 - 손실: 0.2064, 정확도: 0.9966, 시간: 5.44초
--------------------------------------------------
에포크 98/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 98 완료 - 손실: 0.1898, 정확도: 0.9964, 시간: 5.46초
--------------------------------------------------
에포크 99/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 99 완료 - 손실: 0.1722, 정확도: 0.9975, 시간: 5.45초
--------------------------------------------------
에포크 100/100 시작...


  0%|          | 0/11 [00:00<?, ?it/s]

에포크 100 완료 - 손실: 0.1565, 정확도: 0.9975, 시간: 5.46초
에포크 100에 체크포인트 저장: ./checkpoints/transformer/ckpt-28
--------------------------------------------------


###Step 5. 모델 평가하기
---
Step 1에서 선택한 전처리 방법을 고려하여 입력된 문장에 대해서 대답을 얻는 예측 함수를 만듭니다.

In [62]:
# 모델 체크포인트 로드 (학습 이후)
# 저장된 체크포인트가 있으면 최신 체크포인트를 복원
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print(f'최신 체크포인트 복원: {ckpt_manager.latest_checkpoint}')

최신 체크포인트 복원: ./checkpoints/transformer/ckpt-28


In [63]:
# 응답 생성 함수 정의
def evaluate(sentence):
    # 문장 전처리
    sentence = preprocess_sentence(sentence)

    # 문장 토큰화
    encoder_input = tokenizer.encode(sentence)
    encoder_input = [START_TOKEN_ID] + encoder_input + [END_TOKEN_ID]
    encoder_input = tf.expand_dims(encoder_input, 0)

    # 디코더 입력은 시작 토큰으로 시작
    decoder_input = [START_TOKEN_ID]
    output = tf.expand_dims(decoder_input, 0)

    # 자동 회귀(auto-regressive) 방식으로 출력 생성
    for i in range(MAX_LENGTH):
        predictions = transformer(
            {
                'inputs': encoder_input,
                'dec_inputs': output
            },
            training=False
        )

        # 마지막 토큰의 예측에서 다음 토큰 선택
        predictions = predictions[:, -1:, :]  # (batch_size, 1, vocab_size)
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)  # int32로 명시적 변환

        # 종료 토큰이 예측되면 반환
        if tf.equal(predicted_id, END_TOKEN_ID):  # tf.equal 사용
            break

        # 예측된 ID를 디코더 입력에 연결
        output = tf.concat([output, predicted_id], axis=-1)

    # 출력 텍스트로 변환
    output_text = tokenizer.decode([i for i in output[0].numpy() if i < tokenizer.vocab_size])

    return output_text

In [64]:
# 챗봇 응답 함수
def chat_response(sentence):
    result = evaluate(sentence)

    # 특수 토큰 제거 및 정리
    result = result.replace('<start>', '').replace('<end>', '').strip()

    return result

In [65]:
# 몇 가지 질문에 대한 응답 테스트
test_questions = [
    '안녕하세요?',
    '오늘 날씨가 어때요?',
    '당신의 이름은 뭐예요?',
    '취미가 뭐예요?',
    '한국어 잘하시네요'
]

for question in test_questions:
    print('질문:', question)
    print('응답:', chat_response(question))
    print()


질문: 안녕하세요?
응답: 는 신세계그룹이 운영하는 대한민국의 편의점이에요 .

질문: 오늘 날씨가 어때요?
응답: 는 신세계그룹이 운영하는 대한민국의 편의점이에요 .

질문: 당신의 이름은 뭐예요?
응답: 러시아 공사관 러시아 은 한러수호조약이 체결된 1885년에 착공되어 1890년 준공되었던 곳인데요

질문: 취미가 뭐예요?
응답: 많이 만에 다시 오셨네요 네 , 저를 찾아주세요

질문: 한국어 잘하시네요
응답: 여행을 하기에 국내에 다닐 곳이 정말 많죠 .



In [66]:
# 대화형 챗봇 인터페이스
def interactive_chat():
    print("한국어 챗봇과 대화를 시작합니다. '종료'를 입력하면 대화가 종료됩니다.")
    print('=' * 80)

    while True:
        user_input = input('사용자 > ')

        if user_input.lower() == '종료':
            print('챗봇과의 대화를 종료합니다.')
            break

        response = chat_response(user_input)
        print('챗봇 >', response)
        print()

In [67]:
# 대화형 인터페이스 실행
interactive_chat()

한국어 챗봇과 대화를 시작합니다. '종료'를 입력하면 대화가 종료됩니다.
사용자 > 안녕하세요!
챗봇 > 는 신세계그룹이 운영하는 대한민국의 편의점이에요 .

사용자 > 신세계 그룹이요?
챗봇 > 오키나와현 오키나와 본섬 북부 지역 중 산이나 자연이 많이 남아있는 지역입니다 .

사용자 > 오키나와는 어떤 곳인가요
챗봇 > 오키나와현 오키나와 본섬 북부 지역 중 산이나 자연이 많이 남아있는 지역입니다 .

사용자 > 산이나 자연이 많다니 멋지네요
챗봇 > 오키나와현 오키나와 본섬 북부 지역 중 산이나 자연이 많이 남아있는 지역입니다 .

사용자 > 똑같은 말만 하시네요
챗봇 > 여행을 하기에 국내에 다닐 곳이 정말 많죠 .

사용자 > 국내는 뭐가 좋나요
챗봇 > 여행을 계획하고 계시는군요 !

사용자 > 네 국내에 뭐가 좋냐구요
챗봇 > 이탈리아 캄파니아주의 카세르타도에 위치해있으며 나폴리로부터는 북쪽으로 45km 카세르타로부터는 북쪽으로 20km 거리에 있어요

사용자 > 대한민국은요
챗봇 > 여행을 하기에 국내에 다닐 곳이 정말 많죠 .

사용자 > 이탈리아는 뭐가 좋은데요
챗봇 > 여행을 계획하고 계시는군요 !

사용자 > 너무 똑같네요
챗봇 > 는 신세계그룹이 운영하는 대한민국의 편의점이에요 .

사용자 > 어쩌라구요
챗봇 > 여행을 다녀오셨군요 . 재밌으셨다니 다행입니다 .

사용자 > 바보야!
챗봇 > 는 신세계그룹이 운영하는 대한민국의 편의점이에요 .

사용자 > 종료
챗봇과의 대화를 종료합니다.


In [71]:
# 대화형 인터페이스 실행
interactive_chat()

한국어 챗봇과 대화를 시작합니다. '종료'를 입력하면 대화가 종료됩니다.
사용자 > 안녕? 관광/ 감상의 유적에 대해서 이해를 높이고 있던 중에 궁금한 점이 생겨서 왔어!
챗봇 > ! 저는 지명에 대해 잘 알고 있으니 무엇이든 물어봐주세요 !

사용자 > 내가 유적에 대한 책을 읽고 있었는데, 낙덕정에 대해 궁금해졌어. 혹시 낙덕정에 대해 알려줄 수 있을까?
챗봇 > 오키나와현 오키나와 본섬 북부 지역 중 산이나 자연이 많이 남아있는 지역입니다 .

사용자 > 안녕! 유적에 대해서 궁금한 게 있어서 다시 찾아왔어!
챗봇 > ! 3주 동안 잘 지내셨나요 ? 유적에 대해서 궁금한 점이 있으시다면 무엇이든 물어봐주세요 .

사용자 > 
챗봇 > 는 신세계그룹이 운영하는 대한민국의 편의점이에요 .

사용자 > 오! 경암루가 정자를 의미하는 거였구나. 새로운 정보 고마워.
챗봇 > 궁금한 점은 없으신가요 ?

사용자 > 응 어느 지명인지 잘 몰라서 네게 물어보는거야
챗봇 > 는 이탈리아 캄파니아주의 카세르타도에 위치해있으며 나폴리로부터는 북쪽으로 45km 카세르타로부터는 북쪽으로 20km 거리에 있습니다 .

사용자 > 종료
챗봇과의 대화를 종료합니다.


In [68]:
# 디버깅용 함수
def debug_evaluate(sentence):
    # 문장 전처리
    sentence = preprocess_sentence(sentence)

    # 전처리된 문장 확인
    print("전처리된 문장:", sentence)

    # 문장 토큰화
    encoder_input = tokenizer.encode(sentence)
    print("인코더 입력 토큰:", encoder_input)
    print("인코더 입력 토큰 디코딩:", tokenizer.decode(encoder_input))

    encoder_input = [START_TOKEN_ID] + encoder_input + [END_TOKEN_ID]
    encoder_input = tf.expand_dims(encoder_input, 0)

    # 디코더 입력은 시작 토큰으로 시작
    decoder_input = [START_TOKEN_ID]
    output = tf.expand_dims(decoder_input, 0)

    # 각 스텝의 출력 저장
    step_outputs = []

    # 자동 회귀(auto-regressive) 방식으로 출력 생성
    for i in range(MAX_LENGTH):
        predictions = transformer(
            {
                'inputs': encoder_input,
                'dec_inputs': output
            },
            training=False
        )

        # 마지막 토큰의 예측에서 다음 토큰 선택
        predictions = predictions[:, -1:, :]  # (batch_size, 1, vocab_size)
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        # 생성된 토큰 확인
        token_id = predicted_id.numpy()[0][0]
        token_text = tokenizer.decode([token_id]) if token_id < tokenizer.vocab_size else f"<SPECIAL_{token_id}>"
        step_outputs.append((token_id, token_text))

        # 종료 토큰이 예측되면 반환
        if tf.equal(predicted_id, END_TOKEN_ID):
            break

        # 예측된 ID를 디코더 입력에 연결
        output = tf.concat([output, predicted_id], axis=-1)

    # 각 스텝 출력 확인
    print("\n생성된 토큰 시퀀스 (단계별):")
    for i, (token_id, token_text) in enumerate(step_outputs):
        print(f"스텝 {i+1}: ID={token_id}, 텍스트='{token_text}'")

    # 전체 출력 텍스트 확인
    full_output = output[0].numpy()
    print("\n전체 출력 토큰:", full_output)

    # 출력 텍스트로 변환
    output_text = tokenizer.decode([i for i in full_output if i < tokenizer.vocab_size])
    print("최종 출력 텍스트:", output_text)

    return output_text

# 테스트
test_sentence = "안녕하세요?"
debug_result = debug_evaluate(test_sentence)

전처리된 문장: 안녕하세요 ?
인코더 입력 토큰: [37, 4]
인코더 입력 토큰 디코딩: 안녕하세요 ?

생성된 토큰 시퀀스 (단계별):
스텝 1: ID=76, 텍스트='는 '
스텝 2: ID=3648, 텍스트='신세계그룹이 '
스텝 3: ID=1152, 텍스트='운영하는 '
스텝 4: ID=758, 텍스트='대한민국의 '
스텝 5: ID=2074, 텍스트='편의점이에요'
스텝 6: ID=1, 텍스트=' .'
스텝 7: ID=6245, 텍스트='<SPECIAL_6245>'

전체 출력 토큰: [6244   76 3648 1152  758 2074    1]
최종 출력 텍스트: 는 신세계그룹이 운영하는 대한민국의 편의점이에요 .


## 6. 모델 저장 및 불러오기

학습이 끝난 모델을 저장하고 나중에 다시 불러올 수 있도록 함수를 구현합니다.

In [69]:
# 토크나이저 저장
import pickle

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump({
        'tokenizer': tokenizer,
        'start_token': START_TOKEN_ID,
        'end_token': END_TOKEN_ID,
        'vocab_size': VOCAB_SIZE
    }, handle, protocol=pickle.HIGHEST_PROTOCOL)

print('토크나이저 저장 완료')

토크나이저 저장 완료


In [70]:
# 토크나이저 불러오기 함수
def load_tokenizer():
    with open('tokenizer.pickle', 'rb') as handle:
        data = pickle.load(handle)

    return data['tokenizer'], data['start_token'], data['end_token'], data['vocab_size']

# 테스트
loaded_tokenizer, loaded_start, loaded_end, loaded_vocab_size = load_tokenizer()
print(f'불러온 어휘 사전 크기: {loaded_vocab_size}')

불러온 어휘 사전 크기: 6246
